In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import chi2_contingency
from scipy.stats import zscore

# Análise Exploratória Inicial

In [99]:
dados = pd.read_excel("C:\Projetos Pessoais\DataScience\segmentacao_Clientes_Ecomerce\data\Vendas_Varejo_Online.xlsx")

In [100]:
dados.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [101]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [102]:
dados.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [103]:
# Valores NaN
dados.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [104]:
# Dados únicos
dados.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [105]:
dados.shape

(541909, 8)

# Linhas gerais de raciocínio

- invoiceNO que começa com a letra "C" significa cancelamentos
- Para os campos NaN em CostumerID, irei preencher o mesmo código quando a data e hora for igual
- Criação do campo "valor total", multiplicando valor unitario e quantidade
- Criar plotagem no Plotly para Países
- Séries Temporais para as vendas em cada país, expostas no plotly
- Dividir campo data e hora e realizar preenchimento
- Total de 9.291 transações canceladas
- Total de 1.454 de trasanções zeradas com NaN na descrição

# Engenharia de Atributos

- Como o dataset possui muitos erros que impactariam a EDA, farei a engenharia de atributos antes de analisar a fundo a os dados, para corrigir falhas mais visíveis e melhorar o formato dos dados

#### Exclusão dos itens cancelados "C" e "A"

In [106]:
# Exclusão letra 'C' no 'InvoiceNo'

mask = dados['InvoiceNo'].str.contains('C')
mask = np.where(mask.isna(), False, mask)
mask_C = np.where(mask == True, False, True)

In [107]:
dados = dados[mask_C]

In [108]:
dados.shape

(532621, 8)

In [109]:
# Retirando invoiceNo que contenha "A"

mask_A = dados['InvoiceNo'].str.contains('A')
mask_A2 = np.where(mask_A.isna(), False, mask_A)
mask_A3 = np.where(mask_A2 == True, False, True)
dados = dados[mask_A3]

In [110]:
dados.shape

(532618, 8)

In [111]:
dados.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [112]:
dados['InvoiceDate'][0].value

1291191960000000000

In [113]:
dados['InvoiceDate'][100].value

1291196220000000000

In [114]:
dados['InvoiceDate'][2].value

1291191960000000000

In [115]:
dados['InvoiceDate'][0].value == dados['InvoiceDate'][1].value

True

In [116]:
# Dados ausentes na variável Customer
dados['CustomerID'].isna().sum()

134694

In [117]:
dados['CustomerID'].max()

18287.0

#### Preenchendo valores ausentes da seguinte maneira: 
- Se a compra foi realizada na mesma data e horário da compra anterior, subtende-se que foram realizadas pelo mesmo cliente;
- No momento em que o horário mudar, o código de cliente gerado também mudará
- Como o maior código de cliente é 18287, usarei códigos a partir de 19000


In [118]:
dados

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [119]:
def preencher_customer_id(df):
    codigo_cliente = 19000
    
    # Redefinir o índice do DataFrame
    df = df.reset_index(drop=True)
    
    for indice, linha in df.iterrows():
        if pd.isna(linha['CustomerID']):
            if indice == 0 or linha['InvoiceDate'] != df.loc[indice - 1, 'InvoiceDate']:
                codigo_cliente += 1
            df.at[indice, 'CustomerID'] = codigo_cliente
    return df

# Aplicar a função ao DataFrame
dados_customer = preencher_customer_id(dados)

In [120]:
dados['CustomerID'].isna().sum()

134694

In [121]:
dados_customer['CustomerID'].isna().sum()

0

In [122]:
dados.CustomerID[300426]

nan

In [123]:
dados_customer.CustomerID[300426]

14221.0

In [124]:
# Verificando os novos códigos gerados 
dados_customer[dados_customer['CustomerID'] > 18999]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
613,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,19001.0,United Kingdom
1431,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,19002.0,United Kingdom
1432,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,19002.0,United Kingdom
1433,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,19002.0,United Kingdom
1434,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,19002.0,United Kingdom
...,...,...,...,...,...,...,...,...
532249,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,21825.0,United Kingdom
532250,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,21825.0,United Kingdom
532251,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,21825.0,United Kingdom
532252,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,21825.0,United Kingdom


### Retirando os NaN de Descrição, pois em todos o valor unitário do material é 0

In [125]:

mascara = dados_customer['Description'].isna()
mascara2 = np.where(mascara == False, True, mascara)

# Aplicando a mascara
dados_semNA = dados_customer

dados_semNA.dropna(subset=['Description'], inplace=True)
dados_semNA.shape

(531164, 8)

In [126]:
dados_semNA['CustomerID'] = dados_semNA['CustomerID'].astype(int)
dados_semNA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531164 entries, 0 to 532617
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    531164 non-null  object        
 1   StockCode    531164 non-null  object        
 2   Description  531164 non-null  object        
 3   Quantity     531164 non-null  int64         
 4   InvoiceDate  531164 non-null  datetime64[ns]
 5   UnitPrice    531164 non-null  float64       
 6   CustomerID   531164 non-null  int32         
 7   Country      531164 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 34.4+ MB


In [127]:
# Criando Nova coluna com os valores totais de cada transação
dados_semNA['Valor_total'] = dados_semNA['Quantity'] * dados_semNA['UnitPrice']

In [128]:
dados_semNA

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Valor_total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
532613,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
532614,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
532615,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
532616,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


### Criando um novo campo para a hora de venda e deixando apenas o dia da venda no campo invoiceDate

- Objetivo é verificar em quais horários as vendas tem maiores volumes
- Verificar tambem, com gráficos do Plotly, o volume de vendas dos paises por dia

In [131]:
# Campo de horário
dados_semNA['Horario_Venda'] = dados_semNA['InvoiceDate'].dt.time

In [133]:
# Campo de data
dados_semNA['Data_Venda'] = dados_semNA['InvoiceDate'].dt.date
dados_semNA = dados_semNA.drop('InvoiceDate', axis = 1)

In [134]:
dados_semNA.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Valor_total,Horario_Venda,Data_Venda
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850,United Kingdom,15.30,08:26:00,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,3.39,17850,United Kingdom,20.34,08:26:00,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850,United Kingdom,22.00,08:26:00,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850,United Kingdom,20.34,08:26:00,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850,United Kingdom,20.34,08:26:00,2010-12-01


In [135]:
dados_semNA.isna().sum()

InvoiceNo        0
StockCode        0
Description      0
Quantity         0
UnitPrice        0
CustomerID       0
Country          0
Valor_total      0
Horario_Venda    0
Data_Venda       0
dtype: int64